In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/reversible2/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')
%cd /home/schirrmr/


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible.sliced import sliced_from_samples

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
from reversible.plot import create_bw_image
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible.revnet import ResidualBlock, invert, SubsampleSplitter, ViewAs, ReversibleBlockOld
from spectral_norm import spectral_norm
from conv_spectral_norm import conv_spectral_norm

def display_text(text, fontsize=18):
    fig = plt.figure(figsize=(12,0.1))
    plt.title(text, fontsize=fontsize)
    plt.axis('off')
    display(fig)
    plt.close(fig)

In [ ]:
from braindecode.datasets.bbci import BBCIDataset
from braindecode.mne_ext.signalproc import mne_apply
from collections import OrderedDict
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne

def load_file(filename):
    cnt = BBCIDataset(filename).load()
    cnt = cnt.drop_channels(['STI 014'])
    def car(a):
        return a - np.mean(a, keepdims=True, axis=0)

    cnt = mne_apply(
        car, cnt)
    return cnt


def create_set(cnt):
    marker_def = OrderedDict([('Right Hand', [1]), ('Left Hand', [2],),
                             ('Rest', [3]), ('Feet', [4])])
    ival = [500,1500]
    from braindecode.mne_ext.signalproc import mne_apply, resample_cnt
    from braindecode.datautil.signalproc import exponential_running_standardize, bandpass_cnt

    log.info("Resampling train...")
    cnt = resample_cnt(cnt, 250.0)
    log.info("Standardizing train...")
    cnt = mne_apply(lambda a: exponential_running_standardize(a.T ,factor_new=1e-3, init_block_size=1000, eps=1e-4).T,
                         cnt)
    cnt = resample_cnt(cnt, 32.0)
    cnt = resample_cnt(cnt, 64.0)

    dataset = create_signal_target_from_raw_mne(cnt, marker_def, ival)
    return dataset
def create_inputs(dataset):
    x_right = dataset.X[dataset.y == 0]

    x_rest = dataset.X[dataset.y == 2]

    inputs_a = np_to_var(x_right[:,0:1,:,None], dtype=np.float32)

    inputs_b = np_to_var(x_rest[:,0:1,:,None], dtype=np.float32)
    inputs = [inputs_a, inputs_b]
    return inputs


In [ ]:
train_cnt = load_file('/data/schirrmr/schirrmr/HGD-public/reduced/train/4.mat')
train_cnt = train_cnt.reorder_channels(['C3', 'C4'])
train_set = create_set(train_cnt)
train_inputs = create_inputs(train_set)

In [ ]:
test_cnt = load_file('/data/schirrmr/schirrmr/HGD-public/reduced/test/4.mat')
test_cnt = test_cnt.reorder_channels(['C3', 'C4'])
test_set = create_set(test_cnt)
test_inputs = create_inputs(test_set)

In [ ]:
def rev_block(n_c, n_i_c):
     return ReversibleBlockOld(
        nn.Sequential(
        nn.Conv2d(n_c // 2, n_i_c,(3,1), stride=1, padding=(1,0),bias=True),
        nn.ReLU(),
            nn.Conv2d(n_i_c, n_c // 2,(3,1), stride=1, padding=(1,0),bias=True)),
         
        nn.Sequential(
        nn.Conv2d(n_c // 2, n_i_c,(3,1), stride=1, padding=(1,0),bias=True),
        nn.ReLU(),
            nn.Conv2d(n_i_c, n_c // 2,(3,1), stride=1, padding=(1,0),bias=True)))

In [ ]:
from rfft import 

In [ ]:
from rfft import RFFT, Interleave
from discriminator import ProjectionDiscriminator
from reversible.revnet import SubsampleSplitter, ViewAs
from reversible.util import set_random_seeds
from reversible.revnet import init_model_params
from torch.nn import ConstantPad2d
import torch as th
from conv_spectral_norm import conv_spectral_norm
from disttransform import DistTransformResNet


set_random_seeds(2019011641, True)
feature_model_a = nn.Sequential(
    SubsampleSplitter(stride=[2,1],chunk_chans_first=False),# 2 x 32
    rev_block(2,32),
    rev_block(2,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True), # 4 x 16
    rev_block(4,32),
    rev_block(4,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True), # 8 x 8
    rev_block(8,32),
    rev_block(8,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True), # 16 x 4
    rev_block(16,32),
    rev_block(16,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True), # 32 x 2
    rev_block(32,32),
    rev_block(32,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True), # 64 x 1
    rev_block(64,64),
    rev_block(64,64),
    rev_block(64,64),
    rev_block(64,64),
    ViewAs((-1,64,1, 1), (-1,64)),
    RFFT(),
)

In [ ]:
optimizer = th.optim.Adam(feature_model_a.parameters())

In [ ]:
n_epochs = 101
for i_epoch in range(n_epochs):
    outs_0  = feature_model_a(train_inputs[0])
    outs_1  = feature_model_a(train_inputs[1])

    preds_0 = F.log_softmax(outs_0[:,:2], dim=1)
    preds_1 = F.log_softmax(outs_1[:,:2], dim=1)

    labels_0 = th.zeros(len(preds_0), dtype=th.int64)
    labels_1 = th.ones(len(preds_1), dtype=th.int64)

    loss = F.nll_loss(preds_0, labels_0) + F.nll_loss(preds_1, labels_1)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 20) == 0:
        print("Epoch {:d} of {:d}".format(i_epoch, n_epochs))
        print("Loss: {:E}".format(loss.item()))
        for set_name, dataset in (('Train', train_inputs), ('Test', test_inputs)):
            pred_labels_0 = np.argmax(var_to_np(feature_model_a(dataset[0])[:,:2]), axis=1)
            pred_labels_1 = np.argmax(var_to_np(feature_model_a(dataset[1])[:,:2]), axis=1)
            acc = np.mean(np.concatenate(((pred_labels_0 == 0), (pred_labels_1 == 1))))
            print("{:s} Accuracy: {:.2f}%".format(set_name, acc * 100))

In [ ]:
for i_class in range(2):
    outs = feature_model_a(train_inputs[i_class])

    _, i_sorted = th.sort(F.softmax(outs[:,:2], dim=1)[:,i_class])

    sorted_ins = var_to_np(train_inputs[i_class].squeeze())[var_to_np(i_sorted)]

    fig, axes = plt.subplots(8,3, figsize=(16,24), sharex=True, sharey=True)
    for i_ax, ax in enumerate(axes.flatten()):
        ax.plot(sorted_ins[i_ax * 9])
    display(fig)
    plt.close(fig)

In [ ]:
th.mean(feature_model_a(test_inputs[0])[:,:2], dim=0)

In [ ]:
th.mean(feature_model_a(test_inputs[1])[:,:2], dim=0)

In [ ]:
set_inputs = train_inputs
for i_class in range(2):
    with th.no_grad():
        outs = feature_model_a(set_inputs[i_class])
        # only take those confidently correctly predicted
        preds = F.softmax(outs[:,:2], dim=1)
        mask = preds[:,i_class] > 0.9
        outs = outs[mask]
        ins = set_inputs[i_class][mask]
        outs_other = feature_model_a(set_inputs[1-i_class])
        preds_other = F.softmax(outs_other[:,:2], dim=1)
        mask_other = preds_other[:,1-i_class] > 0.9
        outs_other = outs_other[mask_other]
        other_ins = set_inputs[1-i_class][mask_other]
        n_min = min(len(outs_other), len(outs))
        outs = outs[:n_min]
        outs_other = outs_other[:n_min]
        outs.data[:,:2] = outs_other[:,:2].data * 0.02 + outs.data[:,:2] * 0.98
        inverted = invert(feature_model_a, outs, )
        fig, axes = plt.subplots(3,3, figsize=(16,7), sharex=True, sharey=True)
        for i_example in range(3):
            axes[0][i_example].plot(var_to_np(ins[i_example].squeeze()))
            axes[1][i_example].plot(var_to_np(inverted[i_example].squeeze()))
            axes[2][i_example].plot(var_to_np(other_ins[i_example].squeeze()))
        display(fig)
        plt.close(fig)